In [ ]:
#######################
    #### Receve the values from the joystick and send directly to the robot WITH thred #### X, Y, Z, angles, gripper, slow mode


import inputs
from IPython.display import clear_output
from _thread import *
import threading
import sys, time, struct, urx, math, cv2
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper
import numpy as np
import inputs, socket



valx=0
valy=0
valz=0
frames=0
totalframes=2050
valx=0
valy=0
valz=0
open_g = 0
close_g = 0
slow = 0
angl = 0

P = np.array([0.2,0.2,0.2]).reshape(3,1);
V = np.array([0.0,0.0,0.0]).reshape(3,1)
e = np.zeros((3,1));
ea = np.zeros((3,1));

k1 = ( np.diag([3,3,3]));
k2 = (np.diag([0.3,0.3,0.3]));
k3 = ( np.diag([0.001,0.001,0.001]));


k1 = ( np.diag([1.5,1.5,1.5]));
k2 = (np.diag([0.15,0.15,0.15]));
k3 = ( np.diag([0.001,0.001,0.001]));

vmax = 0.3
x_refr = 0.8
y_refr = 0
z_refr = 0.5
gr=0

robot = urx.Robot("192.168.88.175")#, use_rt=True)
robotiqgrip = Robotiq_Two_Finger_Gripper(robot)
s = socket.socket()
print('socket created ')
port = 1234
s.bind(('127.0.0.3', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)

inicial = 1
d=0
i=0
def f():
    global df

    while True :
        
        de=c.recv(1024).decode("utf-8") #Collect data from port and decode into  string

        try :
            de = de.replace(',','.')
            df = []
            for i in de.split(';') :
                df.append(float(i))
            
            xa = df[0]
            ya = df[1]
            za = df[2]
        except:
            print("An exception occurred") 
            pass

p = threading.Thread(target=f, args=())
p.start()
time.sleep(5)    

while True:
            clear_output(wait=True)
            pose = robot.getl()
            xn = float(pose[0])
            yn = float(pose[1])
            zn = float(pose[2])
            rx =float(pose[3])
            ry =float(pose[4])
            rz =float(pose[5])
            print("pose: ", pose)
            
            
            
            if inicial == 1:     # first iteration
                
                posen = robot.getl()
                
                
                #posicion inicial tracker
                x_in = df[0]
                y_in = df[1]
                z_in = df[2]
                
                #posicion inicial robot
                pos_hrx_inicial= xn
                pos_hry_inicial= yn
                pos_hrz_inicial= zn
                time.sleep(3)
                pos_hrx_inicial= xn
                pos_hry_inicial= yn
                pos_hrz_inicial= zn
                
                # inicializo variable auxiliares
                
                x_ant = 0
                y_ant = 0
                z_ant = 0
                
                difx = 0
                dify = 0                
                difz = 0
                       
                inicial = 0
                
            #Second iteration 
            print("pose inicial: ", posen)
            if d==1:
                #difx= df[0] - x_ant
                #dify= df[1] - y_ant
                #difz= df[2] - z_ant
                
                difx= df[0] - x_in
                dify= df[1] - y_in
                difz= df[2] - z_in
                
                #work area limits
                if difx > 0.2:
                    difx=0.2
                elif difx < -0.2:
                    difx=-0.2
                    
                if dify > 0.1:
                    dify=0.1
                elif dify < -0.1:
                    dify=-0.1
                    
                    
                d=0
                
                #difx=0.1
                #dify=-0.10
                difz=0
                
            d+=1
            
            
            print("difx: ", difx)
            print("dify: ", dify)
            print("difz: ", difz)
            #valx = round(stretch(difx, -2, 2, -127, 127))
            #valy = round(stretch(dify, -2, 2, -127, 127))
            #valz = round(stretch(difz, -2, 2, -127, 127))



            #Robotic current position
            P = np.zeros((3,1));
            P[0,0] = float(xn)
            P[1,0] = float(yn)
            P[2,0] = float(zn)
            
            print('Actual Point',P)


            maxval=0.015
            
            #Posicion deseada 
            x_refr = pos_hrx_inicial + difx
            y_refr = pos_hry_inicial + difz
            z_refr = pos_hrz_inicial + dify
            
            #nueva posicion del tracker
            x_ant = df[0]
            y_ant = df[1]
            z_ant = df[2]
            

            #DESIRED POINT
            Pa = np.zeros((3,1));   
            Pa[0,0] = float(x_refr)
            Pa[1,0] = float(y_refr)
            Pa[2,0] = float(z_refr)
            #print('Desired Point',Pa)
            print("Punto de llegada",Pa)
            
            #Controler
            e = Pa - P           
            hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
            ea = e        
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
            
            
            VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
            if (VMAX > vmax):
                vx1 = vx1*vmax/VMAX
                vy1 = vy1*vmax/VMAX
                vz1 = vz1*vmax/VMAX
            
            V[0,0] = vx1
            V[1,0] = vy1
            V[2,0] = vz1

            vx1=round(vx1,3)
            vy1=round(vy1,3)
            vz1=round(vz1,3)
            
            '''
            if vx1<0.1 and vx1>-0.1:
                vx1=0
            if vy1<0.1 and vy1>-0.1:
                vy1=0
            if vz1<0.1 and vz1>-0.1:
                vz1=0
            '''
            print("vx1: ", vx1)
            print("vy1: ", vy1)
            print("vz1: ", vz1)
               
            print("error: ", e)
            if abs(e[0])>0.001 or abs(e[1])>0.001 or abs(e[2])>0.001:
                robot.speedl_tool((-vx1,1*vy1,-vz1,0 ,0, 0 ), acc=0.3, min_time=1)
            else:
                robot.speedl_tool((-vx1*0,vy1*0,vz1*0,0 ,0, 0 ), acc=0.3, min_time=1)
            
            #robot.speedl_tool((-vx1,vy1,-vz1,0 ,0, 0 ), acc=0.3, min_time=1)
            time.sleep(0.005)
            
            i+=1


pose:  [0.04476863998601455, -0.16596817656142063, 0.23021345443755126, 2.2274420958626027, 2.1506213559200273, -0.026971600853452574]
pose inicial:  [0.06459675110182954, -0.27428913058932897, 0.2986567468641195, 2.234389728938765, 2.158719130810803, 0.015014295975908204]
difx:  -0.2
dify:  -0.1
difz:  0
Actual Point [[ 0.04476864]
 [-0.16596818]
 [ 0.23021345]]
Punto de llegada [[-0.13540325]
 [-0.27428913]
 [ 0.19865675]]
vx1:  -0.04
vy1:  -0.024
vz1:  -0.007
error:  [[-0.18017189]
 [-0.10832095]
 [-0.03155671]]
